# GENDER-AGE PREDICTION

### Importing libraries

In [2]:
import cv2
import numpy as np

### Load the pre-trained face detection, gender classification, and age classification models

In [3]:
face_model = cv2.dnn.readNetFromTensorflow('models/opencv_face_detector_uint8.pb', 'models/opencv_face_detector.pbtxt')
gender_model = cv2.dnn.readNetFromCaffe('models/gender_deploy.prototxt', 'models/gender_net.caffemodel')
age_model = cv2.dnn.readNetFromCaffe('models/age_deploy.prototxt', 'models/age_net.caffemodel')

### Define age ranges

In [4]:
age_ranges = ["0-2", "4-6", "8-12", "15-20", "25-32", "38-43", "48-53", "60-100"]

### Function to detect faces in an image

In [8]:
def detect_faces(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the image
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))
    face_model.setInput(blob)
    detections = face_model.forward()
    
    # Extract bounding box coordinates for each detected face
    face_boxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:  # Filter out weak detections
            box = detections[0, 0, i, 3:7] * np.array([image.shape[1], image.shape[0], image.shape[1], image.shape[0]])
            (startX, startY, endX, endY) = box.astype("int")
            face_boxes.append((startX, startY, endX, endY))
    
    return face_boxes

### Function to classify gender for a given face ROI

In [9]:
def classify_gender(image, face_box):
    # Define input shape for gender classification model
    gender_input_shape = (227, 227)
    
    # Resize the input image to match the required input shape of the gender classification model
    resized_image = cv2.resize(image, (gender_input_shape[1], gender_input_shape[0]))
    
    # Preprocess the resized image for gender classification
    gender_input = cv2.dnn.blobFromImage(resized_image, 1.0, (gender_input_shape[1], gender_input_shape[0]), (104.0, 177.0, 123.0))
    
    # Perform gender classification
    gender_model.setInput(gender_input)
    gender_preds = gender_model.forward()
    
    # Print the shape and contents of gender_preds for troubleshooting
    print("Shape of gender_preds:", gender_preds.shape)
    print("Contents of gender_preds:", gender_preds)
    
    # Check if gender_preds has the expected shape
    if gender_preds.shape[0] != 1 or gender_preds.shape[1] != 1:
        print("Invalid shape of gender_preds:", gender_preds.shape)
        return "Unknown"
    
    gender = "Male" if gender_preds[0][0][0] < 0.5 else "Female"
    
    return gender

In [10]:
def classify_age(image, face_box):
    # Define input shape for age classification model
    age_input_shape = (227, 227)

    # Extract the face ROI (Region of Interest) from the image
    (startX, startY, endX, endY) = face_box
    face_roi = image[startY:endY, startX:endX]

    # Resize the face ROI to match the required input shape of the age classification model
    face_input = cv2.resize(face_roi, (age_input_shape[1], age_input_shape[0]))

    # Preprocess the face input for age classification
    face_input = cv2.cvtColor(face_input, cv2.COLOR_BGR2RGB)
    face_input = face_input.astype("float") / 255.0
    face_input = np.transpose(face_input, (2, 0, 1))  # Transpose to channel-first format

    try:
        # Perform age classification
        age_model.setInput(face_input)
        age_preds = age_model.forward()

        # Check if age_preds has the expected shape
        if age_preds.shape[0] != 1 or age_preds.shape[1] != len(age_ranges):
            print("Invalid shape of age_preds:", age_preds.shape)
            return "Unknown"
        age = age_ranges[np.argmax(age_preds)]
    except Exception as e:
        print("Error during age classification:", str(e))
        age = "Unknown"

    return age

###  Function to visualize predictions on the image

In [11]:
def visualize_predictions(image, faces):
    for (startX, startY, endX, endY) in faces:
        cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)

        # Classify gender and age for the current face
        gender = classify_gender(image, (startX, startY, endX, endY))
        age = classify_age(image, (startX, startY, endX, endY))
        
        print("Gender:", gender)
        print("Age:", age)

        # Draw text indicating the predicted gender and age on the image
        text = f"{gender}, {age}"
        cv2.putText(image, text, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    # Display the image with predicted gender and age
    cv2.imshow("Gender and Age Detection", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

###  Main function